## Automated clustering of articles from The 20 newsgroups text dataset

Dataset available from https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

This code retrieves and prepares the dataset, uses SVD to reduce dimensionality, automatically selects the optimal number of clusters (based on evaluation metrics) and peforms clustering before displaying the outputs for interpretation.

In [1]:
#Import modules
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import random
random.seed(39)
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics
from operator import itemgetter

### Load and prepare dataset

In [2]:
#Load in 20 news groups data
newsgroups_data = fetch_20newsgroups(subset = 'all', remove = ('headers','footers','quotes'), shuffle = True)

In [3]:
#Get the data and labels for each article and zip them together, before shuffling
all_news_data = list(zip(newsgroups_data.data, newsgroups_data.target))
random.shuffle(all_news_data)

#Get all data and labels
X_raw = [text for (text, label) in all_news_data]
y_raw = [label for (text, label) in all_news_data]

### Singular Value Decomposition dimensionality reduction

In [4]:
#SVD to reduce dimensionality

#Instantiate vectorizer
#Ignore terms that appear in fewer than 2 documents (too rare to be of value)
#Ignore terms that appear in more than 50% of documents (too common to be of value)
#Ignore English stopwords
vectorizer = TfidfVectorizer(min_df=2, max_df=0.5, stop_words='english', use_idf=True)

#Define function to take input data and transform to data of a given number of dimensions
def transform_data(input_data, output_dimensions):
    vectorized_data = vectorizer.fit_transform(input_data)
    #Instantiate SVD instance of desired dimensions
    svd = TruncatedSVD(output_dimensions)
    pipeline = make_pipeline(svd, Normalizer(copy = False))
    #Reduce dimensionality of data using pipeline
    reduced_data = pipeline.fit_transform(vectorized_data)
    
    return reduced_data, svd

#Reduce dataset
X_reduced, svd = transform_data(X_raw, 300)

### Grid Search to find optimal number of clusters

In [5]:
#Search for the optimal number of clusters for the KMeans algorithm to find
#List to store tuples of (number of clusters, v measure score) 
v_measures = []

#Find v measure score for 2 to 20 clusters
for i in range(2,25):
    #Instantiate a KMeans model
    km_model = KMeans(n_clusters = i, random_state = 20)

    #Fit model to reduced dataset
    km_model = km_model.fit(X_reduced)
    
    #Find v measure score for current number of clusters and append
    v_measure = metrics.v_measure_score(y_raw, km_model.labels_)
    v_measures.append((i, v_measure))

#Optimal number of clusters defined as the number of clusters which gives the highest v measure score
optimal_k = max(v_measures, key = itemgetter(1))[0]

### Train final model

In [6]:
#Instantiate final KMeans model using optimal number of clusters found previously
km_model_final = KMeans(n_clusters = optimal_k, random_state = 39)

#Fit final KMeans model 
km_model_final = km_model_final.fit(X_reduced)

### Evaluate model performance and present results for interpretation

#### Homogeneity Score: Measures the extent to which each cluster only contains members of a single class

$ Homogeneity = 1 - \frac{S(C|K)}{S(C)} $

#### Completeness Score: Measures the extent to which all members of a class are assigned to the same cluster

$ Completeness = 1 - \frac{S(K|C)}{S(K)} $

Where S(A|B) is the conditional entropy of A given B and S(A) is the conditional entropy of A.

$ S(C|K) = - \sum_{c = 1}^{|C|} \sum_{k = 1}^{K} \frac{n_{c,k}}{n} \times log\frac{n_{c,k}}{n_{k}} $

$ S(C|K) = - \sum_{k = 1}^{|K|} \sum_{c = 1}^{C} \frac{n_{c,k}}{n} \times log\frac{n_{c,k}}{n_{c}} $

$ S(C) = - \sum_{c = 1}^{|C|} \frac{n_{c}}{n} \times log\frac{n_{c}}{n} $

$ S(K) = - \sum_{k = 1}^{|K|} \frac{n_{k}}{n} \times log\frac{n_{k}}{n} $

#### V-Measure Score: Harmonic mean of homogeneity and completeness

#### Most discriminative words for esach cluster are presented below for interpretation of the topic of each cluster

In [7]:
#Evaluate performance of final model
homogeneity_score = metrics.homogeneity_score(y_raw, km_model_final.labels_)
completeness_score = metrics.completeness_score(y_raw, km_model_final.labels_)
v_measure_score = metrics.v_measure_score(y_raw, km_model_final.labels_)

print(f'''Model Evaluation
Homogenity Score: {homogeneity_score}
Completeness Score: {completeness_score}
V-Measure Score: {v_measure_score}''')

Model Evaluation
Homogenity Score: 0.32788752966947354
Completeness Score: 0.36287137206862946
V-Measure Score: 0.34449356345889326


In [8]:
#Display the most discriminative words for each cluster

#Reverse dimensionality reduction
unreduced_centroids = svd.inverse_transform(km_model_final.cluster_centers_)

#Order centroids
order_centroids = unreduced_centroids.argsort()[:, ::-1]

#Get terms associated with each vector / feature
terms = vectorizer.get_feature_names_out()

#Print the 50 most discriminative words for each cluster
for i in range(optimal_k):
    print("Cluster " + str(i + 1) + ": ")
    cl_terms = ""
    for ind in order_centroids[i, :50]:
        cl_terms += terms[ind] + " "
    print(cl_terms + "\n")

Cluster 1: 
edu cs university geb pitt n3jxp chastity dsl cadre intellect gordon skepticism shameful banks surrender mail soon mit email internet send uiuc article ftp cc thanks know pub don colorado does david indiana virginia just info au use like lcs cica good 1993 think export information reply apr andrew news 

Cluster 2: 
like just bike space use time good used power way new ve know high don low long right orbit make got light ground earth going thing little engine shuttle need work problem years cost really launch want probably sure does actually big think moon water ll small ride things better 

Cluster 3: 
fbi koresh batf compound children gas did government warrant davidians agents atf waco people bd started evidence tear cult clinton assault said weapons place raid reno building believe press know just think federal david didn like branch illegal don tanks suicide law news inside killed point story search followers time 

Cluster 4: 
people government don just right like thi